# Dunder Data Challenge #3 Solution - (Optimal) - Multiple Custom Grouping Aggregations

In this notebook, I will present an 'optimal' solution to challenge number 3. The original challenge description is re-posted below, followed by the naive and optimal solutions.

## Master Python, Data Science and Machine Learning

[Master the fundamentals of python, data science, and machine learning with my comprehensive and direct path to success.][0] There are over 500 exercises and projects with detailed solutions to help you become an expert.

## Original Challenge

This challenge is going to be fairly difficult, but should answer a question that many pandas users face - What is the best way to do a grouping operation that does many custom aggregations? In this context, a 'custom aggregation' is defined as one that is not directly available to use from pandas and one that you must write a custom function for. 

In Dunder Data Challenge 1, a single aggregation, which required a custom grouping function, was the desired result. In this challenge, you'll need to make several aggregations when grouping. There are a few different solutions to this problem, but depending on how you arrive at your solution, there could arise enormous performance differences. I am looking for a compact, readable solution with very good performance.

### Sales Data

In this challenge, you will be working with some mock sales data found in the sales.csv file. It contains 200,000 rows and 9 columns.

[0]: https://www.dunderdata.com/store

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/sales.csv', parse_dates=['date'])
df.head()

,customer_id,date,country,region,delivery_type,cost_type,duration,revenue,cost
0,13763,2019-03-25,Portugal,F,slow,expert,60,553,295
1,13673,2019-12-06,Singapore,I,slow,experienced,60,895,262
2,10287,2018-09-04,India,I,slow,novice,60,857,260
3,14298,2018-06-21,Morocco,F,fastest,expert,120,741,238
4,11523,2019-01-05,Luxembourg,A,fast,expert,120,942,263


In [3]:
df.shape

(200000, 9)

### Challenge

There are many aggregations that you will need to return and it will take some time to understand what they are and how to return them. The following definitions for two time periods will be used throughout the aggregations.

Period **2019H1** is defined as the time period beginning January 1, 2019 and ending June 30, 2019.
Period **2018H1** is defined as the time period beginning January 1, 2018 and ending June 30, 2018.

### Aggregations
Now, I will list all the aggregations that are expected to be returned. Each bullet point represents a single column. Use the first word after the bullet point as the new column name.

For every country and region, return the following:
* recency: Number of days between today's date (9/9/2019) and the maximum value of the 'date' column 
* fast_and_fastest: Number of unique customer_id in period 2019H1 with delivery_type either 'fast' or 'fastest'
* rev_2019: Total revenue for the period 2019H1
* rev_2018: Total revenue for the period 2018H1
* cost_2019: Total cost for period 2019H1
* cost_2019_exp: Total cost for period 2019H1 with cost_type 'expert'
* other_cost: Difference between cost_2019 and cost_2019_exp
* rev_per_60: Total of revenue when duration equals 60 in period 2019H1 divided by number of unique customer_id when duration equals 60 in period 2019H1 
* profit_margin: Take the difference of rev_2019 and cost_2019_exp then divide by rev_2019. Return as percentage
* cost_exp_per_60: Total of cost when duration is 60 and cost_type is 'expert' in period 2019H1 divided by the number of unique customer_id when duration equals 60 and cost_type is 'expert' in period 2019H1 
* growth: Find the percentage growth from revenue in period 2019H1 compared to the revenue in period 2018H1

##  Solution

### Naive Solution - Custom function with apply

The naive solution was presented in detail in the previous post. The end result is a massive custom function containing many boolean filters used to find specific subsets of data to aggregate. For each group, a Series was returned with 11 values. Each of these values became a new row in the resulting DataFrame. Let's take a look at the custom function:

In [4]:
def f_final(x):
    # filters
    is_2019H1 =        x['date'].between('2019-01-01', '2019-06-30')
    is_2018H1 =        x['date'].between('2018-01-01', '2018-06-30')
    is_fast_fastest =  x['delivery_type'].isin({'fast', 'fastest'})
    is_exp =           x['cost_type'] == 'expert'
    is_60 =            x['duration'] == 60
    is_2019H1_exp =    is_2019H1 & is_exp
    is_2019H1_60 =     is_2019H1 & is_60
    is_2019H1_60_exp = is_2019H1 & is_60 & is_exp
    
    # column calculations
    recency =          (pd.Timestamp('today') - x['date'].max()).days
    fast_and_fastest = x.loc[is_fast_fastest, 'customer_id'].nunique()
    rev_2019 =         x.loc[is_2019H1, 'revenue'].sum()
    rev_2018 =         x.loc[is_2018H1, 'revenue'].sum()
    cost_2019 =        x.loc[is_2019H1, 'cost'].sum()
    cost_2019_exp =    x.loc[is_2019H1_exp, 'cost'].sum()
        
    # helper calculations
    rev_2019_60 =           x.loc[is_2019H1_60, 'revenue'].sum()
    uniq_cust_2019_60 =     x.loc[is_2019H1_60, 'customer_id'].nunique()
    cost_2019_exp_60 =      x.loc[is_2019H1_60_exp, 'cost'].sum()
    uniq_cust_2019_exp_60 = x.loc[is_2019H1_60_exp, 'customer_id'].nunique()
    
    # more column calculations
    other_cost =       cost_2019 - cost_2019_exp
    rev_per_60 =       rev_2019_60 / uniq_cust_2019_60
    profit_margin =    (rev_2019 - cost_2019_exp) / rev_2019 * 100
    cost_exp_per_60 =  cost_2019_exp_60 / uniq_cust_2019_60
    growth =           (rev_2019 - rev_2018) / rev_2018 * 100
    
    d = {
        'recency': recency,
        'fast_and_fastest': fast_and_fastest,
        'rev_2019': rev_2019,
        'rev_2018': rev_2018,
        'cost_2019': cost_2019,
        'cost_2019_exp': cost_2019_exp,
        'other_cost': other_cost,
        'rev_per_60': rev_per_60,
        'profit_margin': profit_margin,
        'cost_exp_per_60': cost_exp_per_60,
        'growth': growth
    }
    
    return pd.Series(d)

Applying this final function and formatting the resulting DataFrame yields the following.

In [5]:
df1 = df.groupby(['country', 'region']).apply(f_final)
df1.head().style.format('{:,.0f}')

Our performance with this final function is more than 3.5 seconds. While this is a lot less than 8 hours, the calculations we performed in the custom function were fairly simple and our data was just 200k rows. If the data and complexity of the custom function increases by an 1-2 order of magnitudes each, hours of computation time await.

In [6]:
%timeit df.groupby(['country', 'region']).apply(f_final)

3.92 s ± 199 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Optimal Solution

In order to greatly increase our performance, we need to take advantage of the **built-in** methods available to groupby objects. Above, we used a custom function to do many, many calculations. These calculations were performed on each group. Let's get the total number of groups.

In [7]:
len(df.groupby(['country', 'region']))

520

For each of these 520 groups, the massive `f_final` function was called, recomputing each filter. Running these same calculations for each group is one of the main causes of poor performance with `apply`.

Take a look at all the filters in `f_final`. You'll notice that each of them are independent on the particular group. This means that we can calculate these filters **before** grouping and get the same result. As a concrete example, take a look at the following DataFrame.

In [8]:
df_rev = pd.DataFrame({'state': ['TX', 'TX', 'TX', 'CA', 'CA', 'CA'], 
                       'category': ['tech', 'energy', 'energy', 'tech', 'energy', 'energy'],
                       'revenue' : [10, 5, 8, 20, 12, 2]})
df_rev

,state,category,revenue
0,TX,tech,10
1,TX,energy,5
2,TX,energy,8
3,CA,tech,20
4,CA,energy,12
5,CA,energy,2


Let's calculate the revenue for each state, but just for the energy category. A naive solution involves writing a custom function, where each group will be filtered for just the energy category and then have the revenue summed.

In [9]:
def f(x):
    is_energy = x['category'] == 'energy'
    return x.loc[is_energy, 'revenue'].sum()

Using this custom function with apply returns the correct revenue for each state's energy category.

In [10]:
df_rev.groupby('state').apply(f)

state
CA    14
TX    13
dtype: int64

Instead, we can create an entire new column for just the energy revenue. First, we create a boolean Series where `True` corresponds to 'energy'. We multiply this Series by the original revenue column. Because `False` evaluates to 0 and `True` evaluates as 1, the new column will be just like the original, but have the 0 everywhere the category is not energy.

In [11]:
filt = df_rev['category'] == 'energy'
df_rev['energy_revenue'] = filt * df_rev['revenue']
df_rev

,state,category,revenue,energy_revenue
0,TX,tech,10,0
1,TX,energy,5,5
2,TX,energy,8,8
3,CA,tech,20,0
4,CA,energy,12,12
5,CA,energy,2,2


We can now use the built-in `sum` method instead of our custom function during the grouping. There is no need for `apply` here.

In [12]:
df_rev.groupby('state')['energy_revenue'].sum()

state
CA    14
TX    13
Name: energy_revenue, dtype: int64

### Use the `where` method
Instead of replacing the filtered values with 0, as we did above, you might need to make them missing. This is crucial if you are calculating something like the mean or median, which will take into account the value of 0. The `where` method will replace the `False` values of the passed boolean Series with NaN.

In [13]:
filt = df_rev['category'] == 'energy'
df_rev['energy_revenue'] = df_rev['revenue'].where(filt)
df_rev

,state,category,revenue,energy_revenue
0,TX,tech,10,NaN
1,TX,energy,5,5.0
2,TX,energy,8,8.0
3,CA,tech,20,NaN
4,CA,energy,12,12.0
5,CA,energy,2,2.0


We can call the same groupby to get the same result.

In [14]:
df_rev.groupby('state')['energy_revenue'].sum()

state
CA    14.0
TX    13.0
Name: energy_revenue, dtype: float64

### Filter with our challenge data
Let's calculate `rev_2019` which is defined as the revenue during the first half of 2019. Let's use the naive way of thinking first by defining a custom function.

In [15]:
def get_rev_2019(x):
    is_2019H1 = x['date'].between('2019-01-01', '2019-06-30')
    return x.loc[is_2019H1, 'revenue'].sum()

In [16]:
df.groupby(['country', 'region']).apply(get_rev_2019).head()

country    region
Argentina  A         150508
           B         139048
           C         118035
           D         131728
           E         146201
dtype: int64

Now, let's use our new method of applying the filter to the entire DataFrame first, creating a new column, and then using the built-in `sum` method.

In [17]:
is_2019H1 = df['date'].between('2019-01-01', '2019-06-30')
df['rev_2019'] =  df['revenue'].where(is_2019H1)
df.groupby(['country', 'region'])['rev_2019'].sum().head()

country    region
Argentina  A         150508.0
           B         139048.0
           C         118035.0
           D         131728.0
           E         146201.0
Name: rev_2019, dtype: float64

### Custom function vs built-in method performance comparison
Let's compare the performance between the two methods.

In [18]:
%timeit df.groupby(['country', 'region']).apply(get_rev_2019)

633 ms ± 15.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%%timeit
is_2019H1 = df['date'].between('2019-01-01', '2019-06-30')
df['rev_2019'] = df['revenue'].where(is_2019H1)
df.groupby(['country', 'region'])['rev_2019'].sum()

27.3 ms ± 530 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


The custom function is 25x slower than the built-in method and this is just a simple calculation. The more complex the custom function, the larger the performance difference becomes.

### The trick to avoiding apply

If you are attempting to avoid using apply, then you have no choice but to use the built-in groupby methods. This limits the possibilities and forces you to approach the problem differently. The main 'trick' is to execute operations to the entire DataFrame before using the groupby method. 

Not all operations will be able to be executed on the entire DataFrame, only those that are **independent** of the group. So, how do you know if an operation is independent of the group? The operation will not have calculation that is specific to the current group. For instance, we are grouping by country and region. If an operation is dependent on the particular country or region, then it would not be able to be executed on the entire DataFrame. 

A concrete example can help here - If the definition of the first half of the year was January through July for Greece and January through June for all other countries, then the calculation of revenue for the first half of the year would depend on the group.

In this challenge, all the operations are independent of the group. There are no special cases based on the group. This means that we can execute all of our operations that we used within the custom function passed to `apply` outside of it before we group.

### Complete optimal solution

The complete optimal solution will now be given. We will use the same definition for our filters as we did in the custom function, but instead calculate them on the entire DataFrame. We will then create new columns that have NaN where the filters is `False`. 

In [20]:
df_new = df.copy()
# filters
is_2019H1 =        df_new['date'].between('2019-01-01', '2019-06-30')
is_2018H1 =        df_new['date'].between('2018-01-01', '2018-06-30')
is_fast_fastest =  df_new['delivery_type'].isin({'fast', 'fastest'})
is_exp =           df_new['cost_type'] == 'expert'
is_60 =            df_new['duration'] == 60
is_2019H1_exp =    is_2019H1 & is_exp
is_2019H1_60 =     is_2019H1 & is_60
is_2019H1_60_exp = is_2019H1 & is_60 & is_exp

# new columns
df_new['rev_2019'] =              df_new['revenue'].where(is_2019H1)
df_new['rev_2018'] =              df_new['revenue'].where(is_2018H1)
df_new['fast_and_fastest'] =      df_new['customer_id'].where(is_fast_fastest)
df_new['cost_2019'] =             df_new['cost'].where(is_2019H1)
df_new['cost_2019_exp'] =         df_new['cost'].where(is_2019H1_exp)
df_new['rev_2019_60'] =           df_new['revenue'].where(is_2019H1_60)
df_new['uniq_cust_2019_60'] =     df_new['customer_id'].where(is_2019H1_60)
df_new['cost_2019_exp_60'] =      df_new['cost'].where(is_2019H1_60_exp)
df_new['uniq_cust_2019_exp_60'] = df_new['customer_id'].where(is_2019H1_60_exp)

df_new.iloc[:5, -10:]

,cost,rev_2019,rev_2018,fast_and_fastest,cost_2019,cost_2019_exp,rev_2019_60,uniq_cust_2019_60,cost_2019_exp_60,uniq_cust_2019_exp_60
0,295,553.0,NaN,NaN,295.0,295.0,553.0,13763.0,295.0,13763.0
1,262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,238,NaN,741.0,14298.0,NaN,NaN,NaN,NaN,NaN,NaN
4,263,942.0,NaN,11523.0,263.0,263.0,NaN,NaN,NaN,NaN


We can now use just the built-in groupby methods to aggregate the data.

In [21]:
df_new1 = \
    df_new.groupby(['country', 'region']).agg(
       recency =               ('date', 'max'),
       fast_and_fastest =      ('fast_and_fastest', 'nunique'),
       rev_2019 =              ('rev_2019', 'sum'),
       rev_2018 =              ('rev_2018', 'sum'),
       cost_2019 =             ('cost_2019', 'sum'),
       cost_2019_exp =         ('cost_2019_exp', 'sum'),
       rev_2019_60 =           ('rev_2019_60', 'sum'),
       uniq_cust_2019_60 =     ('uniq_cust_2019_60', 'nunique'),
       cost_2019_exp_60 =      ('cost_2019_exp_60', 'sum'),
       uniq_cust_2019_exp_60 = ('uniq_cust_2019_exp_60', 'nunique'))

df_new1.head()

recency  fast_and_fastest  rev_2019  rev_2018  cost_2019  \
country   region                                                               
Argentina A      2019-12-04               138  150508.0   82912.0    49577.0   
          B      2019-12-06               143  139048.0   92112.0    46153.0   
          C      2019-12-06               129  118035.0   98472.0    38786.0   
          D      2019-12-06               135  131728.0   79600.0    44190.0   
          E      2019-12-06               177  146201.0   93119.0    49600.0   

                  cost_2019_exp  rev_2019_60  uniq_cust_2019_60  \
country   region                                                  
Argentina A             18553.0      71106.0                 92   
          B             15732.0      67471.0                 90   
          C             12661.0      57721.0                 74   
          D             17217.0      60629.0                 83   
          E             18372.0      73223.0                 98   

                  cost_2019_exp_60  uniq_cust_2019_exp_60  
country   region                                           
Argentina A                10215.0                     41  
          B                 8694.0                     36  
          C                 7747.0                     32  
          D                 7284.0                     30  
          E                10772.0                     43

This is not the final DataFrame, as some columns can only be calculated from the result of the aggregated values. We also need to drop some of these intermediate columns that are no longer desired in the result.

In [22]:
df_new1['recency'] =          (pd.Timestamp('now') - df_new1['recency']).dt.days
df_new1['other_cost'] =       df_new1['cost_2019'] - df_new1['cost_2019_exp']
df_new1['rev_per_60'] =       df_new1['rev_2019_60'] / df_new1['uniq_cust_2019_60']
df_new1['profit_margin'] =    (df_new1['rev_2019'] - df_new1['cost_2019_exp']) / df_new1['rev_2019'] * 100
df_new1['cost_exp_per_60'] =  df_new1['cost_2019_exp_60'] / df_new1['uniq_cust_2019_60']
df_new1['growth'] =           (df_new1['rev_2019'] - df_new1['rev_2018']) / df_new1['rev_2018'] * 100
df_new1 = df_new1.drop(columns=['uniq_cust_2019_60', 'uniq_cust_2019_exp_60', 'rev_2019_60', 'cost_2019_exp_60'])
df_new1.head()

recency  fast_and_fastest  rev_2019  rev_2018  cost_2019  \
country   region                                                             
Argentina A           -78               138  150508.0   82912.0    49577.0   
          B           -80               143  139048.0   92112.0    46153.0   
          C           -80               129  118035.0   98472.0    38786.0   
          D           -80               135  131728.0   79600.0    44190.0   
          E           -80               177  146201.0   93119.0    49600.0   

                  cost_2019_exp  other_cost  rev_per_60  profit_margin  \
country   region                                                         
Argentina A             18553.0     31024.0  772.891304      87.673081   
          B             15732.0     30421.0  749.677778      88.685921   
          C             12661.0     26125.0  780.013514      89.273521   
          D             17217.0     26973.0  730.469880      86.929886   
          E             18372.0     31228.0  747.173469      87.433738   

                  cost_exp_per_60     growth  
country   region                              
Argentina A            111.032609  81.527403  
          B             96.600000  50.955359  
          C            104.689189  19.866561  
          D             87.759036  65.487437  
          E            109.918367  57.004478

In [23]:
df_new1.shape

(520, 11)

Let's verify that the DataFrames are equivalent.

In [24]:
df1 = df.groupby(['country', 'region']).apply(f_final)
df1.equals(df_new1.astype('float'))

True

Let's put all the steps of the optimal solution into a single function, which we can then use to measure performance.

In [25]:
def optimal():
    df_new = df.copy()
    
    # filters
    is_2019H1 =        df_new['date'].between('2019-01-01', '2019-06-30')
    is_2018H1 =        df_new['date'].between('2018-01-01', '2018-06-30')
    is_fast_fastest =  df_new['delivery_type'].isin({'fast', 'fastest'})
    is_exp =           df_new['cost_type'] == 'expert'
    is_60 =            df_new['duration'] == 60
    is_2019H1_exp =    is_2019H1 & is_exp
    is_2019H1_60 =     is_2019H1 & is_60
    is_2019H1_60_exp = is_2019H1 & is_60 & is_exp

    # new columns
    df_new['rev_2019'] =              df_new['revenue'].where(is_2019H1)
    df_new['rev_2018'] =              df_new['revenue'].where(is_2018H1)
    df_new['fast_and_fastest'] =      df_new['customer_id'].where(is_fast_fastest)
    df_new['cost_2019'] =             df_new['cost'].where(is_2019H1)
    df_new['cost_2019_exp'] =         df_new['cost'].where(is_2019H1_exp)
    df_new['rev_2019_60'] =           df_new['revenue'].where(is_2019H1_60)
    df_new['uniq_cust_2019_60'] =     df_new['customer_id'].where(is_2019H1_60)
    df_new['cost_2019_exp_60'] =      df_new['cost'].where(is_2019H1_60_exp)
    df_new['uniq_cust_2019_exp_60'] = df_new['customer_id'].where(is_2019H1_60_exp)

    # built-in aggregations
    df_new1 = \
        df_new.groupby(['country', 'region']).agg(
           recency =               ('date', 'max'),
           fast_and_fastest =      ('fast_and_fastest', 'nunique'),
           rev_2019 =              ('rev_2019', 'sum'),
           rev_2018 =              ('rev_2018', 'sum'),
           cost_2019 =             ('cost_2019', 'sum'),
           cost_2019_exp =         ('cost_2019_exp', 'sum'),
           rev_2019_60 =           ('rev_2019_60', 'sum'),
           uniq_cust_2019_60 =     ('uniq_cust_2019_60', 'nunique'),
           cost_2019_exp_60 =      ('cost_2019_exp_60', 'sum'),
           uniq_cust_2019_exp_60 = ('uniq_cust_2019_exp_60', 'nunique'))
    
    df_new1['recency'] =          (pd.Timestamp('now') - df_new1['recency']).dt.days
    df_new1['other_cost'] =       df_new1['cost_2019'] - df_new1['cost_2019_exp']
    df_new1['rev_per_60'] =       df_new1['rev_2019_60'] / df_new1['uniq_cust_2019_60']
    df_new1['profit_margin'] =    (df_new1['rev_2019'] - df_new1['cost_2019_exp']) / df_new1['rev_2019'] * 100
    df_new1['cost_exp_per_60'] =  df_new1['cost_2019_exp_60'] / df_new1['uniq_cust_2019_60']
    df_new1['growth'] =           (df_new1['rev_2019'] - df_new1['rev_2018']) / df_new1['rev_2018'] * 100
    
    # drop columns not needed in final result
    df_new1 = df_new1.drop(columns=['uniq_cust_2019_60', 'uniq_cust_2019_exp_60', 
                                    'rev_2019_60', 'cost_2019_exp_60'])
    return df_new1

### About 20x faster

The optimal solution is about 20x as fast as the naive solution due to precalculating new columns and only using built-in groupby methods.

In [26]:
%timeit optimal()

201 ms ± 2.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Become a pandas expert

If you are looking to completely master the pandas library and become a trusted expert for doing data science work, check out my book [Master Data Analysis with Python][1]. It comes with over 300 exercises with detailed solutions covering the pandas library in-depth.

[1]: https://www.dunderdata.com/master-data-analysis-with-python